# AudioCraft, MusicGen Model Theory

## Abstract

**AudioCraft** is a free and open-source, high-quality audio and music generation framework, developed by [Meta](https://ai.meta.com). It concludes of three main tools: 
* MusicGen: Generate music from text prompts;
* AudioGen: Generate environmental sounds or sound effects from text prompts;
* EnCodec: Compresses and reconstructs audio with high fidelity [1].

**MusicGen** is a single-stage transformer LM (Language Model), operating with tokens (several streams of compressed discrete music representation). It is used to generate high-quality music (mono & stereo sound), with interpreting descriptive text-prompt [2].

## Introduction

`MusicGen` is a text-to-music model that generates musical pieces based on given text descriptions. The process of generating music requires modeling long-range sequences and using the full frequency spectrum, which differs from speech generation and requires a sampling rate of 44.1 kHz or 48 kHz. Music is composed of harmonies and melodies from various instruments, forming complex structures. The generation process aims to replicate these structures, but due to human sensitivity to disharmony, avoiding melodic errors is crucial [2].

The `MusicGen` model provides a framework for modeling multiple parallel streams of acoustic tokens. This framework enables the generation of stereo audio without additional costs. Unsupervised melody conditioning is used to enhance the controllability of the generated sample, allowing for the creation of music that aligns with a specific harmonic and melodic structure [2].

## Method

`MusicGen` is an autoregressive transformer-based decoder that generates music conditioned on text or melody inputs. It uses the [EnCodec](https://github.com/facebookresearch/encodec) audio tokenizer, which enables high-fidelity audio reconstruction from a low frame rate discrete representation [2].


### Audio Tokenization
`EnCodec` is a convolutional auto-encoder that compresses audio using ***Residual Vector Quantization (RVQ)*** and adversarial reconstruction loss. It reduces the audio into a continuous tensor with a lower frame rate ($f_r \ll f_s$), where $f_r$ is the frame rate and $f_s$ is the sampling rate). This tensor is then quantized into discrete tokens, which are arranged into parallel sequences representing different aspects of the audio. In RVQ, each quantizer encodes the residual errors from the previous one, making the tokens interdependent, with the first codebook being the most important [2].

### Codebook interleaving patterns

<p>
    <img src="images/01.ac_codebook_interleaving_patterns.png" alt="Codebook Interleaving Patterns" width="720" height="480">
</p>

**Exact Flattened Autoregressive Decomposition:** The autoregressive model uses a discrete sequence $U \in \{1, \ldots, M\}^S$, where  $S$ is the sequence length. A special token $U_0 = 0$ is used to indicate the beginning of the sequence. The probability distribution for all $t > 0$ is modeled as:

$\mathbb{P}[U_t|U_{t-1}, \ldots, U_0]$

To predict a sequence $\tilde{U}$ that follows the same distribution, it's defined recursively $\tilde{U}_0 = 0$ and for all $t > 0$:

$p_t(\tilde{U}_{t-1}, \ldots, \tilde{U}_0)$

If distribution can be accurately estimated $U$ can be modeled precisely. `EnCodec` produces $K$ codebooks for each time step, so one method is to flatten these into a single sequence $S = d \cdot f_r \cdot K$. This means predicting the codebooks one by one, but this increases complexity and loses some benefits of the low frame rate $f_r$ [2].


**Inexact Autoregressive Decomposition:** Another approach is to predict some codebooks in parallel. A new sequence with $V_0 = 0$ is defined, where:

$V_{t, k} = Q_{t, k}$

Here, $V$ combines all codebooks at time $t$. The distribution is:

$p_{t, k}(\tilde{V}_{t-1}, \ldots, \tilde{V}_0)$

This parallel approach may not match the exact distribution, especially as errors accumulate. However, it maintains the original frame rate $f_r$, which speeds up training and inference [2].


**Arbitrary Codebook Interleaving Patterns:** An experiment with different codebook patterns to see how they affect performance. A codebook pattern consisits of a sequence $P = (P_0, P_1, \ldots, P_S)$, where:

- $P_0 = \emptyset$
- For all $0 < s \leq S$, $P_s \subset \Omega$, such that $P$ is a partition of $\Omega$

Here, $\Omega = \{(t, k) \mid t \in \{1, \ldots, d \cdot f_r\}, k \in \{1, \ldots, K\}\}$ represents all time step and codebook index pairs.

The tokens are predicted in parallel based on these patterns, such as:

- "*Parallel*" pattern: $P_s = \{(s, k) : k \in \{1, \ldots, K\}\}$
- "*Delay*" pattern: $P_s = \{(s - k + 1, k) : k \in \{1, \ldots, K\}, s - k \geq 0\}$

These patterns help understand better the trade-offs between exact and inexact modeling and the impact of different codebook arrangements [2].

### Model Conditioning
**Text Conditioning:** Given a text description that matches the input audio $X$, a conditioning tensor is created $C \in \mathbb{R}^{T_C \times D}$, where $D$ is the dimension used in the autoregressive model. There are three main methods to represent text for audio generation:
- *Pretrained Text Encoder*;
- *Instruction-Based Language Models*;
- *Joint Text-Audio Representations* [2].

**Melody Conditioning:** Instead of text, conditioning on a melody from another track or from whistling/humming is more natural for music. The melody is controlled by conditioning on both the chromagram and text description. Early tests provided the data that using raw chromagrams can lead to overfitting. To reduce this, an information bottleneck is introduced, that is choosing the dominant time-frequency bin in each time step. This approach is unsupervised, eliminating the supervised data requirement [2].

### 2.4 Model Architecture

**Codebook Projection and Positional Embedding:**  
For a given codebook pattern, only some codebooks are present at each pattern step $P_s$. We extract values from $Q$ based on the indices in $P_s$. Each codebook may appear at most once in $P_s$ or not at all. If present, we use a learned embedding table with $N$ entries and dimension $D$. A special token indicates absent codebooks. We sum the values from the codebooks and add a sinusoidal embedding to encode the current step $s$ [2].

**Transformer Decoder:**  
The input is processed by a transformer with $L$ layers and dimension $D$. Each layer has a causal self-attention block followed by a cross-attention block fed with the conditioning tensor $C$. For melody conditioning, $C$ is added as a prefix. Each layer ends with a fully connected block that transforms the output from $D$ channels to $4 \cdot D$ channels, applies a ReLU activation, and then maps back to $D$ channels. The attention and fully connected blocks use residual skip connections and layer normalization (*pre-norm*) [2].

**Logits Prediction:**  
The transformer decoder's output at pattern step $P_s$ is used to predict logits for values of $Q$ at the indices given by $P_{s+1}$. If a codebook is present, logits are predicted using a specific linear layer from $D$ channels to $N$ channels [2].

## Sources

1. [AudioCraft, Meta Blog](https://ai.meta.com/blog/audiocraft-musicgen-audiogen-encodec-generative-ai-audio/)
2. [Copet, J., Kreuk, F. ,Gat, I., Remez, T., Kant, D., Synnaeve, G., Adi, Y., Défossez, A. (2024)  *Simple and Controllable Music Generation.*](https://doi.org/10.48550/arXiv.2306.05284)